In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd                           # Manipulação de dados
import glob                                   # Importação de dados
import re                                     # Expressões regulares
import itertools                              # Ferramenta iteração
import numpy as np                            # Computação científica
import statsmodels.api as sm                  # Modelagem estatística
import seaborn as sns                         # Visualização de dados
import squarify                               # Visualização de treemaps

import matplotlib.pyplot as plt               # Visualização de dados
import matplotlib
plt.style.use('fivethirtyeight') 

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls                    # Ferramentas do plotly
import ipywidgets as widgets

import folium                                 # Visualização de mapas
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

import warnings                               # Ignorar warnings
warnings.filterwarnings("ignore")

# Data Analytics

In [ ]:
plt.style.use('seaborn-pastel')
plt.rcParams['figure.figsize'] = (10,7)

In [ ]:
covid = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')

In [ ]:
covid.head()

In [ ]:
covid = covid.rename(columns={'Country/Region': 'location'})

In [ ]:
covid.groupby('location')['Confirmed'].sum()

## Visualizating the incrising Covid-19 cases from Brazil

In [ ]:
br = covid[covid['location'] == 'Brazil']
covid[covid['location'] == 'Brazil'][['Date','Confirmed']].plot.bar(figsize=(20,8), x='Date')

In [ ]:
br = br.drop(['Lat', 'Long'], 1)

### Adjusts in date format

In [ ]:
covid['Date'] = pd.to_datetime(covid['Date'], format='%m/%d/%y')
covid['Date']

### The tan curve represents a scenario in which hospital systems becomes inundated with coronavirus patients
![risk](https://laistassets.scprdev.org/i/ec32c7efc8c49bac01d79cb9547d4d6e/5e7123bcb555c5000abe3f2e-eight.jpg)


# Import SUS (Health Unic System) data from Brazil

In [ ]:
sus = pd.read_csv('/kaggle/input/sus-brazil/sus_1000_hab.csv') # available beds in each city from Brazil 2010

In [ ]:
sus = sus.drop(columns=['FID', 'gid', 'Censo','legenda', 'Descrição', 'classe'])

In [ ]:
sus.head()

In [ ]:
sus.describe()

In [ ]:
sus.shape

### Checking the available beds number

In [ ]:
sus['total_leitos'] = (sus['Pop_est_2009']/1000)*sus['razao_leitos_sus_1000_hab']

In [ ]:
sus['total_leitos']

In [ ]:
sus.head()

### Result the sum of all beds avaliable in Brazil 

In [ ]:
sus['total_leitos'].sum()

### Total avaliable beds in each state from Brazil

In [ ]:
sus.groupby('UF')['razao_leitos_sus_1000_hab'].sum().plot.barh()

### Incriase of confimed and death cases

In [ ]:
x = br['Confirmed']
x2 =  br['Deaths']
y = br['Date']
plt.figure(figsize=(25, 10))
plt.plot(y,x, label='Confirmed', color='red');
plt.plot(y,x2, label='Deaths', color='black');
plt.scatter(y, x, color='red')
plt.scatter(y, x2, color='blue')
plt.xlabel("Date")
plt.ylabel("Total Cases")
plt.legend(loc='best')
plt.xticks(rotation=45)
plt.grid()
plt.show()

# Prediting the rise of new confimed case using **ARIMA **Model

In [ ]:
br2 = br[['Date', 'Confirmed']]

In [ ]:
br2.index = pd.to_datetime(br.Date, format='%m/%d/%y')
br2.drop(columns='Date', inplace=True)

In [ ]:
br2 = br2[br2['Confirmed'] > 0]

#### Since P-value is greater than the significance level, let’s difference the series and see how the autocorrelation plot looks like.

- If p-value> 0.05: Series is not stationary and must be manipulated.
- If p-value <= 0.05: Series is stationary and the prediction can be made.

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(br2.Confirmed.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

P-value is 1. then Series is not stationary and must be manipulated.

In [ ]:
# TRANSFORMANDO SÉRIE TEMPORAL
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(br2, model='multiplicative')
g = result.plot()
g.set_figwidth(20)
g.set_figheight(10)

### A series of time series analyzes, including the ARIMA model, is a time series, whether stationary or, without the possibility of a trend.

In [ ]:
# Import Libs

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
# PLOT ACF Correlation

g = plot_acf(br2, title="ACF Correlation")
g.set_figheight(8)
g.set_figwidth(14)

### Analyzing the ACF correlation plot, we find that its value decreases slowly and steadily, this characteristic is typical of a non-stationary time series. 

#### To transform the non-stationary time series into a stationary one, just differentiate it n orders, until the result is a stationary time series.

### The resulting time series after order 1 differentiation is a stationary time series, so we find the first parameter of the ARIMA model, the parameter d = 1.

In addition to informing the seasonality of a time series, the ACF correlation plot can also indicate the best values for the q parameter of the ARIMA model.

In [ ]:
br_diff = br2.diff()

g = plot_acf(br_diff, title="ACF Correlation (d = 1)")
g.set_figheight(8)
g.set_figwidth(14)

In [ ]:
# PLOT CORRELAÇÃO ACF

br_diff2 = br2.diff().diff().diff()

g = plot_acf(br_diff2, title="ACF Correlation (d = 2)")
g.set_figheight(8)
g.set_figwidth(14)

In [ ]:
result = adfuller(br_diff.Confirmed.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

# the serie already is stationary, it's not necessary to apply diferentiation 

In [ ]:
# DEFINE COMBINAÇÕES ENTRE p DE 0 A 3, d = 2, e q = 13
pdq = [(p, 2, 1) for p in range(0, 4)]

# DEFINE P e Q ENTRE 0 e 3
P = Q = range(0, 4)

# DEFINE COMBINAÇÕES ENTRE P, D e Q
seasonal_pdq = [(x[0], 1, x[1], 12) for x in list(itertools.product(P, Q))]

# CALCULANDO O MELHOR PARÂMETRO PARA O MODELO

scores = {}

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(br2,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()
            scores[param, param_seasonal] = results.aic
            
        except:
            continue
            
print("Melhoers parâmetros: ", min(scores, key=scores.get)," AUC: ", min(scores.values()))

# CRIANDO MODELO COM OS MELHOERS PARÂMETROS

In [ ]:
# Make the model with the best paraments

mod = sm.tsa.statespace.SARIMAX(br2,
                                order=(0, 2, 1),
                                seasonal_order=(0, 1, 2, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

In [ ]:
# VALIDAÇÃO DO MODELO: PREVENDO OCORRÊNCIAS A PARTIR DE 2017

pred = results.get_prediction(start=pd.to_datetime('2020-02-29'), dynamic=False)
pred_ci = pred.conf_int()

ax = br2.plot(figsize=(14, 8))
pred.predicted_mean.plot(ax=ax, label='predict', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_title("Train to predite growing curve", fontsize=18)
ax.set_xlabel('Date')
ax.set_ylabel('Number of Cases')
plt.legend()

plt.show()

# CALCULANDO MSE

y_forecasted = pred.predicted_mean
y_truth = br2["2020-02-29":].squeeze()

mse = ((y_forecasted - y_truth) ** 2).mean()

print('Mean Squared Error {}'.format(round(mse)))

In [ ]:
# PREVENDO OCORRÊNCIAS PARA OS PRÓXIMOS TRÊS ANOS

pred_uc = results.get_forecast(steps=300)
pred_ci = pred_uc.conf_int()

ax = br2.plot(figsize=(14, 8))
pred_uc.predicted_mean.plot(ax=ax, label='predict')
ax.fill_between(pred_ci.index,pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='k', alpha=.25)

ax.set_title("Prediction of increase in new coronavirus confirming cases in 300 days", fontsize=18)
ax.set_xlabel('Date')
ax.set_ylabel('Number of news cases')

plt.legend()
plt.show()

In [ ]:
xp = pred_uc.predicted_mean.index.tolist()
ylimit = []
for dt in br['Date']:
    ylimit.append(33000)
for dt in xp:
    ylimit.append(33000)
yp = pred_uc.predicted_mean

In [ ]:
ax.fill_between(pred_ci.index,pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='k', alpha=.25)

In [ ]:

trace = [go.Scatter(x = br['Date'], y = br['Confirmed'], name='Total of Cases')]
trace += [go.Scatter(x = xp,y = ylimit, name='Total of beds')]
trace += [go.Scatter(x = xp,y = yp, name='mean rise case')]
trace += [go.Scatter(x = pred_ci.index,y = pred_ci.iloc[:, 0], fill='tonexty', name='min')]
trace += [go.Scatter(x = pred_ci.index,y = pred_ci.iloc[:, 1], fill='tonexty', name='max')]

layout = dict(
    title='Total of coronavirus cases - Brazil',
    yaxis=dict(
    title='new cases'
    ),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=12,
                     label='1yr',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)
fig = dict(data=trace, layout=layout)
py.iplot(fig)

## 2 April will have 10000 case

# Conclusion:
### On April 16, the number of confirmed cases will exceed the number of beds